In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import keras
from keras.models import Sequential,Model
from keras.optimizers import Adam
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


In [29]:
x=list()
y=list()
f=open("data/rock/rock.txt",'r')
r=int(f.read())
f.close();
f=open("data/paper/paper.txt",'r')
p=int(f.read())
f.close();
f=open("data/scissors/scissors.txt",'r')
s=int(f.read())
f.close();

In [30]:
for i in range(r):
    img=cv2.imread("data/rock/%d.jpg"%i,1)
    img = img.reshape(64, 64, 3)
    img=img.tolist()
    x.append(img)
    y.append(0)
for i in range(s):
    img=cv2.imread("data/scissors/%d.jpg"%i,1)
    img = img.reshape(64, 64, 3)
    img=img.tolist()
    x.append(img)
    y.append(2)
for i in range(p):
    img=cv2.imread("data/paper/%d.jpg"%i,1)
    img = img.reshape(64, 64, 3)
    img=img.tolist()
    x.append(img)
    y.append(1)
x=np.asarray(x,dtype='uint8')
y=np.asarray(y)
ycat=to_categorical(y)

In [4]:
def conv_block(X,f,s,filters):
    f1,f2,f3=filters
    
    Xcopy=X
    
    X=Conv2D(filters=f1,kernel_size=(1,1),strides=(s,s))(X)
    X=BatchNormalization(axis=3)(X)
    X=Activation('relu')(X)
    
    X=Conv2D(filters=f2,kernel_size=(f,f),strides=(1,1),padding="same")(X)
    X=BatchNormalization(axis=3)(X)
    X=Activation('relu')(X)
    
    X=Conv2D(filters=f3,kernel_size=(1,1),strides=(1,1),padding="valid")(X)
    X=BatchNormalization(axis=3)(X)
    
    Xcopy=Conv2D(filters=f3,kernel_size=(1,1),strides=(s,s),padding="valid")(X)
    Xcopy=BatchNormalization(axis=3)(X)
    
    X=Add()([X,Xcopy])
    X=Activation('relu')(X)
    
    return X

In [5]:
def identity_block(X,filters,f):
    f1,f2,f3=filters
    
    Xcopy=X
    
    X=Conv2D(filters=f1,kernel_size=(1,1),strides=(1,1),padding="valid")(X)
    X=BatchNormalization(axis=3)(X)
    X=Activation("relu")(X)
    
    X=Conv2D(filters=f2,kernel_size=(f,f),strides=(1,1),padding="same")(X)
    X=BatchNormalization(axis=3)(X)
    X=Activation("relu")(X)
    
    X=Conv2D(filters=f3,kernel_size=(1,1),strides=(1,1),padding="valid")(X)
    X=BatchNormalization(axis=3)(X)
    
    X=Add()([X,Xcopy])
    X=Activation("relu")(X)
    
    return X

In [77]:
def resnet():
    Xin=Input((64,64,3))
    
    X=ZeroPadding2D((3,3))(Xin)
    
    X=Conv2D(64,(7,7),strides=(2,2))(X)
    X=BatchNormalization(axis=3)(X)
    X=Activation('relu')(X)
    X=MaxPooling2D((3,3),strides=(2,2))(X)
    
    X=conv_block(X,3,1,[64,64,256])
    X=identity_block(X,[64,64,256],3)
    X=identity_block(X,[64,64,256],3)
    
    X=conv_block(X,3,2,[128,128,512])
    X=identity_block(X,[128,128,512],3)
    X=identity_block(X,[128,128,512],3)
    X=identity_block(X,[128,128,512],3)
    
    X=conv_block(X,3,2,[256,256,1024])
    X=identity_block(X,[256,256,1024],3)
    X=identity_block(X,[256,256,1024],3)
    X=identity_block(X,[256,256,1024],3)
    X=identity_block(X,[256,256,1024],3)
    X=identity_block(X,[256,256,1024],3)
    
    X=conv_block(X,3,1,[512,512,2048])
    X=identity_block(X,[512,512,2048],3)
    X=identity_block(X,[512,512,2048],3)
    
    X=AveragePooling2D((2,2))(X)
    
    X=Flatten()(X)
    X=Dense(3, activation='softmax')(X)
    
    model=Model(inputs=Xin,outputs=X)
    
    model.compile(Adam(lr=0.00001), loss='categorical_crossentropy',metrics=['accuracy'])
    
    return model

In [78]:
model=resnet()
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
zero_padding2d_10 (ZeroPadding2 (None, 70, 70, 3)    0           input_10[0][0]                   
__________________________________________________________________________________________________
conv2d_478 (Conv2D)             (None, 32, 32, 64)   9472        zero_padding2d_10[0][0]          
__________________________________________________________________________________________________
batch_normalization_478 (BatchN (None, 32, 32, 64)   256         conv2d_478[0][0]                 
__________________________________________________________________________________________________
activation

In [79]:
fo=model.fit(x,ycat,epochs=5,verbose=1,shuffle=True,batch_size=50)

Epoch 1/5
513/513 [==============================] - 31s 61ms/step - loss: 1.3508 - acc: 0.4035
Epoch 2/5
513/513 [==============================] - 5s 10ms/step - loss: 1.0130 - acc: 0.5497
Epoch 3/5
513/513 [==============================] - 5s 10ms/step - loss: 0.6764 - acc: 0.7232
Epoch 4/5
513/513 [==============================] - 5s 10ms/step - loss: 0.6270 - acc: 0.7836
Epoch 5/5
513/513 [==============================] - 5s 10ms/step - loss: 0.4760 - acc: 0.8713


In [82]:
for i in range(0,171):
    q=cv2.imread("data/scissors/%d.jpg"%i,1)
    q=q.reshape(1,64,64,3)
    pred=model.predict(q)
    pred=pred[0]
    pred.tolist()
    m=max(pred)
    if(pred[0]==m):
        print("Rock")
    elif(pred[1]==m):
        print("Paper")
    elif(pred[2]==m):
        print("Scissors")

Scissors
Scissors
Scissors
Scissors
Paper
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Paper
Scissors
Scissors
Scissors
Rock
Scissors
Scissors
Scissors
Scissors
Rock
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Rock
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Scissors
Rock
Scissors
Scissors
Sciss

In [ ]:
model.save("data/traintemp.h5")